In [1]:
# you might have to install transfomers and torch
import torch
from transformers import pipeline


In [2]:
my_model = pipeline('fill-mask', model='bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
my_model('steve jobs is the [MASK] of apple')

[{'score': 0.5164636969566345,
  'token': 3910,
  'token_str': 'founder',
  'sequence': 'steve jobs is the founder of apple'},
 {'score': 0.36049818992614746,
  'token': 5766,
  'token_str': 'ceo',
  'sequence': 'steve jobs is the ceo of apple'},
 {'score': 0.049299854785203934,
  'token': 2343,
  'token_str': 'president',
  'sequence': 'steve jobs is the president of apple'},
 {'score': 0.02111203595995903,
  'token': 8543,
  'token_str': 'creator',
  'sequence': 'steve jobs is the creator of apple'},
 {'score': 0.008550191298127174,
  'token': 2269,
  'token_str': 'father',
  'sequence': 'steve jobs is the father of apple'}]

In [4]:
from slashml import ModelDeployment
import time

In [5]:
# Replace `API_KEY` with your SlasML API token.
client = ModelDeployment(api_key="68b50f46b97f8c0f98fe419df240a22af3da392f")
# client = ModelDeployment(api_key="e7e33ba9a41c1426a425496319a14cab2b3f794f")

In [6]:
response = client.deploy(model_name='my_model_3', model=my_model, requirements=['torch', 'transformers'])

In [7]:
response

{'id': '653adbbd-fb58-47bb-b47b-6586f10739c2',
 'created': '2023-10-10T22:39:56.607079Z',
 'status': 'IN_PROGRESS',
 'name': 'my_model_3'}

In [1]:
client.status(model_version_id=response.id)

NameError: name 'client' is not defined

In [9]:
status = client.status(model_version_id=response.id)

while status.status != 'READY':
    print(f'status: {status.status}')
    print('trying again in 5 seconds')
    time.sleep(5)
    status = model.status(model_version_id=response.id)

    if status.status in ['FAILED', "ERROR"]:
        raise Exception(f'Model deployment {status}')

print(f'model succeeded')

status: IN_PROGRESS
trying again in 5 seconds


KeyboardInterrupt: 

In [12]:
input_text = 'Steve jobs is the [MASK] of Apple.'
prediction = client.predict(model_version_id=response.id, model_input=input_text)

In [ ]:
prediction['model_response']['predictions']